# Networking

In [1]:
%discover
%connect test-argon

test-argon                     serial:///dev/cu.usbmodem1413101        
test-stm32                     serial:///dev/cu.usbmodem208E307542521  
test-stm32-cop                 serial:///dev/cu.usbserial-0143520E     
Connected to test-argon @ serial:///dev/cu.usbmodem1413101


## Check IP Address

In [1]:
import network

ip = network.WLAN(network.STA_IF).ifconfig()[0]
print("IP address: ", ip)

IP address:  10.39.40.140


## Requests

In [1]:
from requests import get
from secrets import openweathermap_apiid

url = "http://api.openweathermap.org/data/2.5/find?lat=38.5&lon=-120&cnt=1&appid={}".format(openweathermap_apiid)

j = get(url).json()
for k,v in j.get('list')[0].items():
    print("{:8} {}".format(k,v))

clouds   {'all': 89}
sys      {'country': 'US'}
snow     None
coord    {'lat': 38.4646, 'lon': -120.04}
rain     None
wind     {'speed': 0.46, 'deg': 125}
dt       1613178462
weather  ({'id': 804, 'description': 'overcast clouds', 'icon': '04d', 'main': 'Clouds'},)
main     {'pressure': 1019, 'temp_min': 273.71, 'feels_like': 272.62, 'temp': 274.91, 'humidity': 89, 'temp_max': 276.48}
id       5326968
name     Bear Valley


## Webserver

In [1]:
import socket
import network

ip = network.WLAN(network.STA_IF).ifconfig()[0]

s = socket.socket()
ai = socket.getaddrinfo("0.0.0.0", 80)
addr = ai[0][-1]

s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
s.bind(addr)
s.listen(2)

print("waiting for connection at http://{}".format(ip))
for i in range(6):
    ss, addr = s.accept()
    print("connected from", addr)
    req = ss.recv(1024)
    if i < 4:
        ss.send('Hello from Airlift, serving request # {}!'.format(i+1).encode())
    else:
        ss.send(b'So long! Server shutting down.')
    ss.close()
s.close()

print("DONE")

waiting for connection at http://10.39.40.140
connected from ('10.39.40.114', 52674)
connected from ('10.39.40.114', 52675)
connected from ('10.39.40.114', 52676)
connected from ('10.39.40.114', 52677)
connected from ('10.39.40.114', 52678)
connected from ('10.39.40.114', 52679)
DONE


### Socket Echo Test

Run [echo server](https://realpython.com/python-sockets/#echo-server) on (some) host (e.g. from a Python3 notebook).

In [ ]:
import socket
import os

N = 512
SERVER = "10.39.40.114"
PORT = 65432

resp = bytearray(N)

print("create socket ...")
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

try:
    s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    ai = socket.getaddrinfo(SERVER, PORT)
    addr = ai[0][-1]

    print("connect to", addr)
    s.connect(addr)
    
    for i in range(5):
        data = bytes(os.urandom(N))
        s.sendall(data)
        s.readinto(resp)
        assert resp == data, "Mismatch!"
except Exception as e:
    print("*****", e)
finally:
    print("Success!")
    s.close()

## GC

In [1]:
from urpc import *
import gc, sys
gc_  = import_('gc')
sys_ = import_('sys')

host = sys.platform
cop  = sys_.get_('platform')

def report():
    print("{:8}  free = {:6}".format(host, gc.mem_free()))
    print("{:8}  free = {:6}".format(cop, gc_.mem_free()))
    reg = registry_()
    print("  ESP32 registry [{}]:".format(len(reg)))
    for k,v in reg.items():
        print("    [{:3}] {}".format(k, v))
        
report()
print("\n>>>>>>> collect ...\n")
gc.collect()
gc_.collect()
report()

nRF52840  free = 136560
esp32     free =  94192
  ESP32 registry [13]:
    [  9] <socket>
    [ 16] <module 'boot' from 'boot.py'>
    [  2] <module 'urequests' from 'urequests.py'>
    [ 15] <socket>
    [  4] 1613178462
    [  5] <module 'usocket'>
    [ 19] <module 'gc'>
    [  6] <module 'usocket'>
    [  7] <module 'network'>
    [ 21] <module 'gc'>
    [ 23] <module 'gc'>
    [ 24] <module 'sys'>
    [ 22] <module 'sys'>

>>>>>>> collect ...

nRF52840  free = 137360
esp32     free =  97552
  ESP32 registry [10]:
    [  5] <module 'usocket'>
    [  9] <socket>
    [  2] <module 'urequests' from 'urequests.py'>
    [ 23] <module 'gc'>
    [ 24] <module 'sys'>
    [ 15] <socket>
    [  6] <module 'usocket'>
    [  7] <module 'network'>
    [ 16] <module 'boot' from 'boot.py'>
    [  4] 1613178462
